In [3]:
import os
import operator
import requests
from typing import TypedDict, List, Annotated
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Load Environment Variables
load_dotenv()

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not SERPER_API_KEY or not OPENAI_API_KEY:
    raise ValueError("❌ Missing API Keys in .env file")

# ==========================================================
# 🤖 LLM & Tools Setup
# ==========================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

@tool
def web_search(query: str) -> str:
    """Web search using Serper.dev."""
    print(f"      🔎 [TOOL EXEC] Searching Serper for: '{query}'")
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    payload = {"q": query, "num": 3}
    try:
        r = requests.post(url, json=payload, headers=headers)
        results = r.json().get("organic", [])[:2]
        return str(results)
    except Exception as e:
        return f"Error: {e}"

tools = [web_search]

# ==========================================================
# 🧠 Prompts
# ==========================================================
RENEWABLE_PROMPT = "Search for 2024 trends in renewable energy adoption. Summarize in 1 sentence."
EV_PROMPT = "Search for 2024 breakthroughs in solid state batteries for EVs. Summarize in 1 sentence."
CARBON_PROMPT = "Search for 2024 updates on Direct Air Capture (DAC) projects. Summarize in 1 sentence."

SYNTHESIS_PROMPT = """
You are a technical writer. Compile the following research into a brief Markdown report.

## Renewable Energy
{renewable}

## Electric Vehicles
{ev}

## Carbon Capture
{carbon}

## Conclusion
Write a one-sentence synthesis of how these technologies intersect.
"""

# ==========================================================
# 📦 Graph State
# ==========================================================
class GraphState(TypedDict):
    # 1. Independent Message Histories (The Fix for Infinite Loops)
    #    We use operator.add to append new messages to the list
    renewable_msgs: Annotated[List, operator.add]
    ev_msgs: Annotated[List, operator.add]
    carbon_msgs: Annotated[List, operator.add]

    # 2. Final Data Stores
    renewable: str
    ev: str
    carbon: str
    final_report: str

# ==========================================================
# 🏭 Node Factory Functions
# ==========================================================

# --- 1. LLM Node Factory ---
def create_research_node(key_name: str, prompt: str):
    def node(state: GraphState):
        print(f"\n🤖 [{key_name.upper()} BRANCH] LLM Checking Context...")
        
        # Load history for THIS specific branch
        history = state.get(key_name, [])
        
        # Logic: If history is empty, send prompt. If not, send full history (prompt + tool result).
        if not history:
            messages = [HumanMessage(content=prompt)]
        else:
            messages = history

        # Bind tools and invoke
        resp = llm.bind_tools(tools).invoke(messages)
        
        if resp.tool_calls:
            print(f"   👉 Decision: Call Tool ({resp.tool_calls[0]['name']})")
        else:
            print(f"   👉 Decision: Done (Content generated)")
            
        return {key_name: [resp]}
    return node

# --- 2. Tool Node Factory ---
def create_tool_node(key_name: str):
    def node(state: GraphState):
        # Extract the last message (which contains the tool call)
        history = state.get(key_name, [])
        last_msg = history[-1]
        
        # Run tool
        tool_executor = ToolNode(tools)
        # ToolNode expects dict with 'messages' key
        result = tool_executor.invoke({"messages": [last_msg]})
        
        print(f"   ✅ Tool Result obtained.")
        # Return result appended to specific branch history
        return {key_name: result['messages']}
    return node

# --- 3. Router Factory ---
def create_router(key_name: str):
    def _router(state: GraphState):
        msgs = state.get(key_name, [])
        last_msg = msgs[-1]
        
        if hasattr(last_msg, "tool_calls") and len(last_msg.tool_calls) > 0:
            return "call_tool"
        return "done"
    return _router

# ==========================================================
# 🟢 Storage Nodes
# ==========================================================
def store_renewable(state: GraphState):
    content = state["renewable_msgs"][-1].content
    print("\n💾 [STORE] Renewable data saved.")
    return {"renewable": content}

def store_ev(state: GraphState):
    content = state["ev_msgs"][-1].content
    print("\n💾 [STORE] EV data saved.")
    return {"ev": content}

def store_carbon(state: GraphState):
    content = state["carbon_msgs"][-1].content
    print("\n💾 [STORE] Carbon data saved.")
    return {"carbon": content}

# ==========================================================
# 🟣 Join & Synthesis Logic
# ==========================================================

def join_node(state: GraphState):
    """
    Dummy node that acts as a gate. 
    It returns an empty dict because logic happens in the Conditional Edge.
    """
    return {}

def should_synthesize(state: GraphState):
    """
    Checks if all 3 parallel branches have finished writing to state.
    """
    print("   👀 [JOIN CHECK] Checking if all data is ready...")
    
    r = state.get("renewable")
    e = state.get("ev")
    c = state.get("carbon")
    
    if r and e and c:
        print("   🚀 All branches complete! Proceeding to Synthesis.")
        return "synthesis"
    
    print("   ⏳ Waiting for other branches...")
    return "wait"

def synthesis_node(state: GraphState):
    print("\n📝 [SYNTHESIS] Generating Final Report...")
    prompt = SYNTHESIS_PROMPT.format(
        renewable=state["renewable"],
        ev=state["ev"],
        carbon=state["carbon"]
    )
    resp = llm.invoke(prompt)
    return {"final_report": resp.content}

# ==========================================================
# 🕸️ Graph Construction
# ==========================================================
workflow = StateGraph(GraphState)

# --- Renewable Branch ---
workflow.add_node("r_llm", create_research_node("renewable_msgs", RENEWABLE_PROMPT))
workflow.add_node("r_tool", create_tool_node("renewable_msgs"))
workflow.add_node("r_store", store_renewable)

workflow.add_edge("r_tool", "r_llm")
workflow.add_conditional_edges("r_llm", create_router("renewable_msgs"), 
                               {"call_tool": "r_tool", "done": "r_store"})

# --- EV Branch ---
workflow.add_node("e_llm", create_research_node("ev_msgs", EV_PROMPT))
workflow.add_node("e_tool", create_tool_node("ev_msgs"))
workflow.add_node("e_store", store_ev)

workflow.add_edge("e_tool", "e_llm")
workflow.add_conditional_edges("e_llm", create_router("ev_msgs"), 
                               {"call_tool": "e_tool", "done": "e_store"})

# --- Carbon Branch ---
workflow.add_node("c_llm", create_research_node("carbon_msgs", CARBON_PROMPT))
workflow.add_node("c_tool", create_tool_node("carbon_msgs"))
workflow.add_node("c_store", store_carbon)

workflow.add_edge("c_tool", "c_llm")
workflow.add_conditional_edges("c_llm", create_router("carbon_msgs"), 
                               {"call_tool": "c_tool", "done": "c_store"})

# --- Start & Join ---
workflow.add_node("start", lambda x: {}) # Dummy start node
workflow.add_edge("start", "r_llm")
workflow.add_edge("start", "e_llm")
workflow.add_edge("start", "c_llm")

# All stores point to the Join Node
workflow.add_node("join", join_node)
workflow.add_edge("r_store", "join")
workflow.add_edge("e_store", "join")
workflow.add_edge("c_store", "join")

# Join Node decides: Synthesis or End (Wait)
workflow.add_node("synthesis", synthesis_node)
workflow.add_edge("synthesis", END)

workflow.add_conditional_edges(
    "join", 
    should_synthesize,
    {
        "synthesis": "synthesis",
        "wait": END 
    }
)

workflow.set_entry_point("start")
app = workflow.compile()

# ==========================================================
# 🏃 Execution
# ==========================================================
if __name__ == "__main__":
    print("🚀 Starting Parallel Research Graph...\n")
    try:
        result = app.invoke({}, config={"recursion_limit": 50})
        
        print("\n" + "#"*50)
        print("📊 FINAL RESEARCH REPORT")
        print("#"*50 + "\n")
        print(result["final_report"])
        
    except Exception as e:
        print(f"\n❌ Execution Failed: {e}")

🚀 Starting Parallel Research Graph...


🤖 [CARBON_MSGS BRANCH] LLM Checking Context...

🤖 [EV_MSGS BRANCH] LLM Checking Context...

🤖 [RENEWABLE_MSGS BRANCH] LLM Checking Context...
   👉 Decision: Call Tool (web_search)
   👉 Decision: Call Tool (web_search)
   👉 Decision: Call Tool (web_search)
      🔎 [TOOL EXEC] Searching Serper for: '2024 breakthroughs in solid state batteries for EVs'
      🔎 [TOOL EXEC] Searching Serper for: '2024 trends in renewable energy adoption'
      🔎 [TOOL EXEC] Searching Serper for: '2024 updates on Direct Air Capture (DAC) projects'
   ✅ Tool Result obtained.   ✅ Tool Result obtained.

   ✅ Tool Result obtained.

🤖 [EV_MSGS BRANCH] LLM Checking Context...

🤖 [RENEWABLE_MSGS BRANCH] LLM Checking Context...

🤖 [CARBON_MSGS BRANCH] LLM Checking Context...
   👉 Decision: Done (Content generated)
   👉 Decision: Done (Content generated)
   👉 Decision: Done (Content generated)

💾 [STORE] Carbon data saved.

💾 [STORE] EV data saved.

💾 [STORE] Renewable data sav